# 10 - Retriever 检索器

## 用途
学习使用 LangChain Retriever 封装向量库检索逻辑，通过参数控制检索结果数量和质量

## 学习目标
- 理解检索器配置参数和作用
- 掌握检索结果数量控制方法
- 能设置检索参数优化结果
- 掌握不同检索策略的使用

## 🔑 前置要求
**注意**：需要先完成 Embedding + Vector Stores 学习，配置 DashScope API

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. 基础 Retriever 创建和配置

In [ ]:
# 基础 Retriever 创建和配置 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔍 基础 Retriever 创建和配置测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
    print("   请先完成 Embedding + Vector Stores 学习")
else:
    try:
        # 初始化 Embeddings
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        # 准备测试数据
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库",
            "RAG 技术结合检索和生成，提高回答准确性",
            "向量相似度计算使用余弦相似度或欧氏距离"
        ]
        
        print(f"📚 准备文档数量: {len(documents)}")
        
        # 创建向量库
        vector_store = FAISS.from_texts(documents, embeddings)
        print(f"✅ 向量库创建完成: {vector_store.index.ntotal} 个文档")
        
        # 创建基础 Retriever
        retriever = vector_store.as_retriever()
        
        print(f"\n🔧 Retriever 基础配置:")
        print(f"   检索器类型: {type(retriever)}")
        print(f"   默认参数: k=4 (默认检索数量)")
        
        # 测试基础检索
        test_query = "如何开发人工智能应用"
        print(f"\n🧪 测试查询: {test_query}")
        
        results = retriever.invoke(test_query)
        
        print(f"\n📊 检索结果:")
        print(f"   结果数量: {len(results)}")
        
        for i, doc in enumerate(results, 1):
            print(f"   结果 {i}: {doc.page_content}")
        
        # 验证点：基础 Retriever 正常工作
        assert len(results) > 0, "检索结果为空"
        assert all(hasattr(doc, 'page_content') for doc in results), "结果缺少内容"
        print(f"\n✅ 验证通过：基础 Retriever 正常工作")
        
    except Exception as e:
        print(f"\n❌ Retriever 创建失败: {e}")

## 2. 检索参数 k 的控制测试

In [ ]:
# 检索参数 k 的控制测试 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📊 检索参数 k 的控制测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化组件
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库",
            "RAG 技术结合检索和生成，提高回答准确性",
            "向量相似度计算使用余弦相似度或欧氏距离"
        ]
        
        vector_store = FAISS.from_texts(documents, embeddings)
        
        # 测试不同的 k 值
        k_values = [1, 2, 3, 5, 7]
        test_query = "机器学习和编程"
        
        print(f"🧪 测试查询: {test_query}")
        print(f"\n📈 不同 k 值的检索结果:")
        
        results_summary = {}
        
        for k in k_values:
            print(f"\n🔍 k = {k}:")
            
            # 创建指定 k 值的 Retriever
            retriever = vector_store.as_retriever(search_kwargs={"k": k})
            
            # 执行检索
            results = retriever.invoke(test_query)
            
            print(f"   实际结果数: {len(results)}")
            
            # 记录结果
            results_summary[k] = {
                "count": len(results),
                "contents": [doc.page_content for doc in results]
            }
            
            # 显示结果
            for i, doc in enumerate(results, 1):
                print(f"     {i}. {doc.page_content}")
        
        # 分析 k 值控制效果
        print(f"\n📊 k 值控制效果分析:")
        print(f"{'k 值':<6} {'结果数':<8} {'控制效果':<12}")
        print("-" * 30)
        
        for k, summary in results_summary.items():
            expected = k
            actual = summary["count"]
            control_status = "✅ 精确" if actual == expected else f"⚠️  {actual}/{expected}"
            print(f"{k:<6} {actual:<8} {control_status:<12}")
        
        # 测试边界情况
        print(f"\n🧪 边界情况测试:")
        
        # k 大于文档总数
        large_k = len(documents) + 5
        large_retriever = vector_store.as_retriever(search_kwargs={"k": large_k})
        large_results = large_retriever.invoke(test_query)
        
        print(f"   k = {large_k} (大于文档总数):")
        print(f"   实际结果数: {len(large_results)} (最多可用文档数)")
        
        # 验证点：能正确控制检索结果数量
        for k in [1, 2, 3]:
            retriever = vector_store.as_retriever(search_kwargs={"k": k})
            results = retriever.invoke(test_query)
            assert len(results) <= k, f"k={k} 时结果数超过限制"
        
        print(f"\n✅ 验证通过：检索结果数量控制正确")
        
    except Exception as e:
        print(f"\n❌ k 参数控制测试失败: {e}")

## 3. 多种检索策略对比

In [ ]:
# 多种检索策略对比 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔄 多种检索策略对比测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化组件
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库",
            "RAG 技术结合检索和生成，提高回答准确性",
            "向量相似度计算使用余弦相似度或欧氏距离"
        ]
        
        vector_store = FAISS.from_texts(documents, embeddings)
        test_query = "人工智能编程技术"
        
        print(f"🧪 测试查询: {test_query}")
        
        # 1. 相似度检索 (默认)
        print(f"\n📊 1. 相似度检索 (Similarity Search):")
        similarity_retriever = vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        similarity_results = similarity_retriever.invoke(test_query)
        
        for i, doc in enumerate(similarity_results, 1):
            print(f"   {i}. {doc.page_content}")
        
        # 2. 最大边际相关性检索 (MMR)
        print(f"\n🔄 2. 最大边际相关性检索 (MMR):")
        try:
            mmr_results = vector_store.max_marginal_relevance_search(
                test_query, 
                k=3, 
                fetch_k=5
            )
            
            for i, doc in enumerate(mmr_results, 1):
                print(f"   {i}. {doc.page_content}")
        except Exception as e:
            print(f"   MMR 检索失败: {e}")
            mmr_results = []
        
        # 3. 阈值检索
        print(f"\n🎯 3. 相似度阈值检索:")
        try:
            threshold_results_with_scores = vector_store.similarity_search_with_score(
                test_query, 
                k=5,
                score_threshold=0.3
            )
            
            for i, (doc, score) in enumerate(threshold_results_with_scores, 1):
                print(f"   {i}. {doc.page_content} (相似度: {score:.4f})")
                
        except Exception as e:
            print(f"   阈值检索失败: {e}")
        
        # 4. 结果对比分析
        print(f"\n📈 检索策略对比分析:")
        print(f"{'策略':<15} {'结果数':<8} {'特点':<20}")
        print("-" * 45)
        
        strategies = [
            ("相似度检索", len(similarity_results), "最相似，可能重复"),
            ("MMR 检索", len(mmr_results), "多样性，减少重复"),
            ("阈值检索", len(threshold_results_with_scores), "质量过滤，数量不定")
        ]
        
        for strategy, count, feature in strategies:
            print(f"{strategy:<15} {count:<8} {feature:<20}")
        
        # 验证点：不同检索策略工作正常
        assert len(similarity_results) > 0, "相似度检索无结果"
        print(f"\n✅ 验证通过：多种检索策略正常工作")
        
    except Exception as e:
        print(f"\n❌ 检索策略对比失败: {e}")

## 4. 检索性能和质量评估

In [ ]:
# 检索性能和质量评估 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
import time

# 加载环境变量
load_dotenv()

print("📏 检索性能和质量评估测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化组件
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        # 扩展测试数据集
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库",
            "RAG 技术结合检索和生成，提高回答准确性",
            "向量相似度计算使用余弦相似度或欧氏距离",
            "深度学习是机器学习的一个重要分支",
            "自然语言处理是 AI 的核心应用领域",
            "数据预处理是机器学习流程的关键步骤"
        ]
        
        vector_store = FAISS.from_texts(documents, embeddings)
        
        # 测试查询集
        test_queries = [
            "人工智能编程",
            "机器学习算法",
            "数据处理技术",
            "向量检索方法"
        ]
        
        print(f"📚 测试数据: {len(documents)} 个文档")
        print(f"🧪 测试查询: {len(test_queries)} 个")
        
        # 性能测试
        print(f"\n⚡ 检索性能测试:")
        
        k_values = [1, 3, 5, 10]
        performance_results = {}
        
        for k in k_values:
            retriever = vector_store.as_retriever(search_kwargs={"k": k})
            
            # 测试多次取平均
            times = []
            for query in test_queries:
                start_time = time.time()
                results = retriever.invoke(query)
                end_time = time.time()
                times.append(end_time - start_time)
            
            avg_time = sum(times) / len(times)
            performance_results[k] = {
                "avg_time": avg_time,
                "result_count": k
            }
            
            print(f"   k={k:<2}: 平均 {avg_time*1000:.1f} ms/查询")
        
        # 质量评估
        print(f"\n📊 检索质量评估:")
        
        # 使用 k=3 进行质量测试
        retriever = vector_store.as_retriever(search_kwargs={"k": 3})
        
        quality_metrics = {
            "total_queries": len(test_queries),
            "total_results": 0,
            "unique_results": set(),
            "avg_result_length": 0
        }
        
        all_lengths = []
        
        for query in test_queries:
            results = retriever.invoke(query)
            quality_metrics["total_results"] += len(results)
            
            for doc in results:
                quality_metrics["unique_results"].add(doc.page_content)
                all_lengths.append(len(doc.page_content))
        
        quality_metrics["unique_count"] = len(quality_metrics["unique_results"])
        quality_metrics["avg_result_length"] = sum(all_lengths) / len(all_lengths) if all_lengths else 0
        quality_metrics["diversity_rate"] = quality_metrics["unique_count"] / quality_metrics["total_results"]
        
        print(f"   总查询数: {quality_metrics['total_queries']}")
        print(f"   总结果数: {quality_metrics['total_results']}")
        print(f"   唯一结果数: {quality_metrics['unique_count']}")
        print(f"   多样性比率: {quality_metrics['diversity_rate']:.1%}")
        print(f"   平均结果长度: {quality_metrics['avg_result_length']:.1f} 字符")
        
        # 验证点：检索性能和质量评估完成
        assert quality_metrics["total_results"] > 0, "无检索结果"
        print(f"\n✅ 验证通过：检索性能和质量评估完成")
        
    except Exception as e:
        print(f"\n❌ 检索性能评估失败: {e}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 Retriever 学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ 基础检索器：vector_store.as_retriever()",
    "✅ 参数控制：search_kwargs={'k': N}",
    "✅ 检索策略：相似度、MMR、阈值",
    "✅ 性能评估：速度、多样性、相关性",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Retriever 最佳实践:")
print("1. 参数设置：根据应用场景选择合适的 k 值")
print("2. 检索策略：相似度检索适合精确匹配，MMR 适合多样性")
print("3. 性能优化：控制 k 值大小，避免过度检索")
print("4. 质量监控：定期评估检索效果和调整参数")
print("5. 成本控制：合理设置检索数量减少后续处理成本")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 RAG 系统完整流程")
print("2. 掌握 Chain 类型：RetrievalQA、ConversationalRetrievalChain")
print("3. 学习记忆管理：ConversationBufferMemory")
print("4. 探索高级检索：多向量检索、混合检索")
print("5. 实践生产级 RAG 应用开发")

# 最终验证：确保 Retriever 基础功能可用
try:
    dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
    
    if dashscope_api_key:
        # 简单测试
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        test_docs = ["测试文档1", "测试文档2", "测试文档3"]
        vector_store = FAISS.from_texts(test_docs, embeddings)
        retriever = vector_store.as_retriever(search_kwargs={"k": 2})
        
        results = retriever.invoke("测试")
        
        print(f"\n🎉 最终验证成功:")
        print(f"   API 状态: 可用")
        print(f"   检索结果: {len(results)} 个文档")
        print(f"   检索器类型: {type(retriever).__name__}")
        print("\n✅ Retriever 学习完成！")
        
        print(f"\n🎊 RAG 数据层学习完成！")
        print(f"   已掌握: Document Loaders → Text Splitters → Embeddings → Vector Stores → Retrievers")
        print(f"   下一步: 学习 RAG 应用层和编排层技术")
        
    else:
        print(f"\n⚠️  最终验证跳过: DashScope API Key 未配置")
        print("   请配置 DASHSCOPE_API_KEY 后重试")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")